In [54]:
# Dependencies
import pandas as pd
import numpy as np
import os

# All Time Roster File Path 
roster_path = os.path.join("..","..","data","rosters","all_time.csv" )

# FF Historical Results Table Path
ff_path = os.path.join('frozen_four_table.csv')


# Open as DFs
all_time_roster_df = pd.read_csv(roster_path)
all_time_roster_df.head()

ff_df = pd.read_csv(ff_path)
ff_df.head()


C:\Users\jbanc\AppData\Local\Temp\ipykernel_1736\1420337596.py:14: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  all_time_roster_df = pd.read_csv(roster_path)


,Year,Champion,Runner_up,Third,Fourth,Site
0,1948,Michigan,Dartmouth,Other semifinalists: BC and Colorado College,Other semifinalists: BC and Colorado College,"Col. Springs, Colo."
1,1949,Boston College,Dartmouth,Michigan,Colorado College,"Col. Springs, Colo."
2,1950,Colorado College,Boston University,Michigan,Boston College,"Col. Springs, Colo."
3,1951,Michigan,Brown,Boston University,Colorado College,"Col. Springs, Colo."
4,1952,Michigan,Colorado College,Yale,St. Lawrence,"Col. Springs, Colo."


## Team Color and Logo Info & Library

In [55]:

# Team Color Information
            # path to table with hex codes
team_color_path = os.path.join('..','..', 'NCAA', 'data', 'team_color_book.csv')
team_colors = pd.read_csv(team_color_path)
highlight_colors = team_colors

# Logo Folder path
logo_folder = '../../images/logos/'

# Dictionary to map team names to their logo filenames
logo_mapping = {
    
    'Michigan State': 'msu.png',
    'Michigan Tech': 'mtu.png',
    'Boston College': 'bc_.png',
    'Michigan': 'mic.png',
    'Minnesota': 'min.png',
    "Wisconsin": "wis.png",
    'RIT': 'rit.png',
    'Quinnipiac': 'qui.png',
    'Cornell': 'cor.png',
    'Boston University': 'bu_.png',
    'Maine': 'mne.png',
    'Massachusetts': 'uma.png',
    'Denver': 'den.png',
    'North Dakota': 'ndk.png',
    'Omaha': 'uno.png',
    'Western Michigan': 'wmu.png'    
}





# ABBRIV LIBRARIES AND WHATNOT

In [56]:
## State and Province Abbreviation library
canada_provinces = {'Ont.':'Ontario', 'B.C.':'British Columbia', 'Alb.':'Alberta', 'Que.':'Quebec', 'Man.':'Manitoba', 'Sask.': 'Saskatchewan', 'MB':'Manitoba', 'Sask':'Saskatchewan', 'N.S.':'Nova Scotia', 'N.B.':'New Brunswick', 'NB.': 'New Brunswick', 'P.E.I.':'Prince Edward Island', 'Newf.':'Newfoundland and Labrador',
                    'N.S.':'Nova Scotia', 'N.B.':'New Brunswick', 'NB.': 'New Brunswick', 'P.E.I.':'Prince Edward Island', 'Newf.':'Newfoundland and Labrador', 'YT.':'Yukon', 'N.W.T.':'Northwest Territories',
                    'Nun.':'Nunavut', 'NWT.':'Northwest Territories'}

us_states = {'Ala.':'Alabama', 'Alaska':'Alaska', 'Ariz.':'Arizona', 'Ark.':'Arkansas', 'Calif.':'California', 'Colo.':'Colorado', 'Conn.':'Connecticut', 'Del.':'Delaware', 'Fla.':'Florida',
                'Ga.':'Georgia', 'Hawaii':'Hawaii', 'Idaho':'Idaho', 'Ill.':'Illinois', 'Ind.':'Indiana', 'Iowa':'Iowa', 'Kan.':'Kansas', 'Ky.':'Kentucky', 'La.':'Louisiana', 'Maine':'Maine',
                'Md.':'Maryland', 'Mass.':'Massachusetts', 'Mich.':'Michigan', 'Minn.':'Minnesota', 'Miss.':'Mississippi', 'Mo.':'Missouri', 'Mont.':'Montana', 'Neb.':'Nebraska', 'Nev.':'Nevada',
                'N.H.':'New Hampshire', 'N.J.':'New Jersey', 'N.M.':'New Mexico', 'N.Y.':'New York', 'N.C.':'North Carolina', 'N.D.':'North Dakota', 'Ohio':'Ohio', 'Okla.':'Oklahoma', 'Ore.':'Oregon',
                'Pa.':'Pennsylvania', 'R.I.':'Rhode Island', 'S.C.':'South Carolina', 'S.D.':'South Dakota', 'Tenn.':'Tennessee', 'Tx':'Texas', 'Tex.':'Texas', 'Tex':'Texas', 'Utah':'Utah', 'Vt.':'Vermont', 'Va.':'Virginia',
                'Wash.':'Washington', 'W.Va.':'West Virginia', 'Wis.':'Wisconsin', 'Wisc.':'Wisconsin', 'Wyo.':'Wyoming'}
             

In [57]:
def preprocess_hometown(hometown):
    try:
        # Initial split by comma
        parts = [part.strip() for part in hometown.split(',')]
        
        # If more than 2 parts, assume the last part is the country or state/province
        # and combine the rest as the city name
        if len(parts) > 2:
            city = ', '.join(parts[:-1])
            state_province_country = parts[-1]
        else:
            city, state_province_country = parts
        
        # Normalize state/province for known abbreviations
        if state_province_country in us_states:
            state_province = us_states[state_province_country]
            country = 'USA'
        elif state_province_country in canada_provinces:
            state_province = canada_provinces[state_province_country]
            country = 'Canada'
        else:
            # Default handling for unexpected formats or international entries
            state_province = None
            country = state_province_country
        
        return city, state_province, country
    except Exception as e:
        # Log error or handle unexpected formats
        print(f"Error processing '{hometown}': {e}")
        return hometown, None, None  # Fallback return values



In [58]:
# Start by copying the data to avoid modifying the original dataframe inadvertently
location_df = all_time_roster_df.copy()

# drop nan values
location_df = location_df.dropna(subset=['Hometown'])
# Apply preprocessing to each entry in the dataframe
location_df['Processed'] = location_df['Hometown'].apply(preprocess_hometown)

# Expand the processed tuples into separate columns
location_df[['City', 'State/Province', 'Country']] = pd.DataFrame(location_df['Processed'].tolist(), index=location_df.index)



In [59]:

### REDUNDANT CODE
# # Split the Hometown column at the comma
location_df[['City', 'State_Province_Country']] = location_df['Processed'].str.split(', ', expand=True)

# Normalize state and province abbreviations and identify countries directly
def normalize_location_data(row, us_states, canada_provinces):
    loc2 = row['State_Province_Country']
    
    # Normalize state and province abbreviations
    if loc2 in us_states:
        row['State/Province'] = us_states[loc2]
        row['Country'] = 'USA'
    elif loc2 in canada_provinces:
        row['State/Province'] = canada_provinces[loc2]
        row['Country'] = 'Canada'
    else:
        # Handle special cases and direct country assignments
        if loc2 == 'Tx' or loc2 == 'MB' or loc2 == 'Yukon':
            special_cases = {
                'Tx': ('Texas', 'USA'),
                'MB': ('Manitoba', 'Canada'),
                'Yukon': ('Yukon', 'Canada'),
                'AUT': (None, 'Austria')
            }
            row['State/Province'], row['Country'] = special_cases.get(loc2, (None, loc2))
        elif row['Player'] == 'Tyrone Bronte':  # Specific player correction
            row['Country'] = 'Australia'
        else:
            row['State/Province'] = None
            row['Country'] = loc2  # Assume the Loc2 is the country for international players not caught by special cases

    return row

# Apply normalization and assignments
location_df = location_df.apply(normalize_location_data, us_states=us_states, canada_provinces=canada_provinces, axis=1)

# Drop the intermediate column as it's no longer needed
location_df.drop(columns='State_Province_Country', inplace=True)

# Clean up the DataFrame to ensure all strings are stripped of leading/trailing whitespace
location_df = location_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Preview the cleaned and updated location data
location_df[['Player', 'City', 'State/Province', 'Country']].sample(10)


ValueError: Columns must be same length as key

In [ ]:
all_time_roster_df.head()

In [ ]:
## How many entries in the Hometown COlumn has 1 comma, how many have 2 commas, do any have 3 commas or 0?
## We are looking for the format of City, State/Province, Country

## Split the Hometown Column into City, State/Province, Country
all_time_roster_df['Hometown'].str.split(',').str.len().value_counts()

# Split the Hometown Column into City, State/Province, Country
all_time_roster_df[['City', 'State/Province', 'Country']] = all_time_roster_df['Hometown'].str.split(',', expand=True)

# Remove leading and trailing white spaces
all_time_roster_df['City'] = all_time_roster_df['City'].str.strip()
all_time_roster_df['State/Province'] = all_time_roster_df['State/Province'].str.strip()
all_time_roster_df['Country'] = all_time_roster_df['Country'].str.strip()

# Look at the unique values in the State/Province Column
all_time_roster_df['Country'].unique()

### Sperate and clean Hometown column

In [ ]:
# Borrowed from Player_per_capita.ipynb

# Rename roster df to use in this legacy code
full_tourney_roster = all_time_roster_df
# full_tourney_roster[['Loc2', 'Loc2', 'Loc3']] = full_tourney_roster['Hometown'].str.split(',', expand=True)

# Split the Hometown column at the commas into 2 or 3 columns
# full_tourney_roster[['Loc1', 'Loc2']] = full_tourney_roster['Hometown'].str.split(',', expand=True)

# If the Loc2 column is is 'Alaska' add a period to the end of the Loc2 column
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x == 'Alaska' else x)
# Same with Ohio, Iowa, Maine
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x in ['Ohio', 'Iowa', 'Maine', 'Utah', 'Texas'] else x)

# Same for canada provinces with NB, NWT and YT
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: x + '.' if x in ['NB', 'NWT', 'YT'] else x)

# If Loc2 does not have a . it is a country and should be moved to a new Country Column, Loc2 COlumn should be cleared
full_tourney_roster['Country'] = full_tourney_roster['Loc2'].apply(lambda x: x if isinstance(x, str) and '.' not in x else None)

# replace Loc2 columns without '.' with an empty string
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].apply(lambda x: '' if isinstance(x, str) and '.' not in x else x)



# Country Value Count
full_tourney_roster['Country'].value_counts()

# Apply the substitution dictionary to the Loc2 column - add countries to the Country column while doing the substitution
def assign_country_and_loc(row):
    # Check if the Loc2 value is in the canada_provinces dictionary
    if row['Loc2'] in canada_provinces:
        # Replace the Loc2 value
        row['Loc2'] = canada_provinces[row['Loc2']]
        # Assign 'Canada' to the Country column
        row['Country'] = 'Canada'
    # Check if the Loc2 value is in the us_states dictionary
    elif row['Loc2'] in us_states:
        # Replace the Loc2 value
        row['Loc2'] = us_states[row['Loc2']]
        # Assign 'USA' to the Country column
        row['Country'] = 'USA'
    # If the Loc2 value is not in either dictionary, do not change the Country value
    return row

# Strip the City, State, and COuntry column of any leading or trailing whitespace
full_tourney_roster['Loc1'] = full_tourney_roster['Loc1'].str.strip()
full_tourney_roster['Loc2'] = full_tourney_roster['Loc2'].str.strip()


# Apply the function to each row
full_tourney_roster = full_tourney_roster.apply(assign_country_and_loc, axis=1)



# full_tourney_roster.sample(20)

full_tourney_roster['Loc2'].value_counts()
# full_tourney_roster['Country'].value_counts()
location_df = full_tourney_roster[['Team', 'Player', 'Position', 'Loc1', 'Loc2', 'Country']]
# Rename Loc1 and Loc2 to City and State/Province
location_df = location_df.rename(columns={'Loc1': 'City', 'Loc2': 'State/Province'})
# Strip the City, State, and COuntry column of any leading or trailing whitespace
location_df['City'] = location_df['City'].str.strip()
location_df['State/Province'] = location_df['State/Province'].str.strip()
location_df['Country'] = location_df['Country'].str.strip()

# If the State/Province is Alaska. set it to Alaska
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Alaska' if x == 'Alaska.' else x)
# Add USA to the Country column for Alaska
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Alaska' else x['Country'], axis=1)
# Same for Iowa and Ohio and Utah and Texas
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Texas' if x == 'Texas.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Texas' else x['Country'], axis=1)


location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Utah' if x == 'Utah.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Utah' else x['Country'], axis=1)


location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Idaho' if x == 'Idaho.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Idaho' else x['Country'], axis=1)

location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Iowa' if x == 'Iowa.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Iowa' else x['Country'], axis=1)
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Ohio' if x == 'Ohio.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Ohio' else x['Country'], axis=1)
# Same for Maine
location_df['State/Province'] = location_df['State/Province'].apply(lambda x: 'Maine' if x == 'Maine.' else x)
location_df['Country'] = location_df.apply(lambda x: 'USA' if x['State/Province'] == 'Maine' else x['Country'], axis=1)




location_df.head(20)